In [1]:
caso = ["piedra", "tijeras", "papel"]

In [2]:
def jugando(p1, p2):
    if p1 == p2:
        resultado = 0
    
    elif p1 == "piedra" and p2 == "tijeras":
        resultado = 1
    elif p1 == "piedra" and p2 == "papel":
        resultado = 2
    elif p1 == "tijeras" and p2 == "piedra":
        resultado = 2
    elif p1 == "tijeras" and p2 == "papel":
        resultado = 1
    elif p1 == "papel" and p2 == "piedra":
        resultado = 1
    elif p1 == "papel" and p2 == "tijeras":
        resultado = 2
        
    return resultado

In [3]:
jugando("piedra", "tijeras")

1

In [4]:
test = [
    ["piedra", "piedra", 0],
    ["piedra", "tijeras", 1],
    ["piedra", "papel", 2]
]

for partida in test:
    print("player1: %s player2: %s Winner: %s Validation: %s" % (
        partida[0], partida[1], jugando(partida[0], partida[1]), partida[2]
    ))

player1: piedra player2: piedra Winner: 0 Validation: 0
player1: piedra player2: tijeras Winner: 1 Validation: 1
player1: piedra player2: papel Winner: 2 Validation: 2


In [5]:
from random import choice
def get_choice():
    return choice(caso)

In [6]:
for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print("player1: %s player2: %s Winner: %s " % (
        player1, player2, jugando(player1, player2)
    ))

player1: papel player2: piedra Winner: 1 
player1: piedra player2: tijeras Winner: 1 
player1: papel player2: piedra Winner: 1 
player1: piedra player2: papel Winner: 2 
player1: piedra player2: piedra Winner: 0 
player1: piedra player2: tijeras Winner: 1 
player1: papel player2: tijeras Winner: 2 
player1: tijeras player2: tijeras Winner: 0 
player1: papel player2: tijeras Winner: 2 
player1: piedra player2: piedra Winner: 0 


In [7]:
def paso_lista(caso):
    if caso=="piedra":
        res = [1,0,0]
    elif caso=="tijeras":
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

data_X = list(map(paso_lista, ["piedra", "tijeras", "papel"]))
data_y = list(map(paso_lista, ["papel", "piedra", "tijeras"]))

print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [8]:
from sklearn.neural_network import MLPClassifier

In [9]:
clf = MLPClassifier(verbose=False, warm_start=True)

In [10]:
model = clf.fit([data_X[0]], [data_y[0]])
print(model)

MLPClassifier(warm_start=True)


/Users/avillanme.com/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [11]:
def aprendiendo_jugando(iters=10, debug=False):
    score = {"win": 0, "loose": 0}
    
    data_X = []
    data_y = []
    
    for i in range(iters):
        player1 = get_choice()
        
        predict = model.predict_proba([paso_lista(player1)])[0]
        
        if predict[0] >= 0.95:
            player2 = caso[0]
        elif predict[1] >= 0.95:
            player2 = caso[1]
        elif predict[2] >= 0.95:
            player2 = caso[2]
        else:
            player2 = get_choice()
            
        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))
        
        winner = jugando(player1, player2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % winner)
        
        if winner==2:
            data_X.append(paso_lista(player1))
            data_y.append(paso_lista(player2))
            
            score["win"]+=1
        else:
            score["loose"]+=1
        
    return score, data_X, data_y

In [12]:
score, data_X, data_y = aprendiendo_jugando(1, debug=True)
print(data_X)
print(data_y)
print("Score: %s %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"]))))
if len(data_X):
    model = model.partial_fit(data_X, data_y)

Player1: papel Player2 (modelo): [0.15812111 0.1502746  0.80772077] --> piedra
Comprobamos: p1 VS p2: 1
[]
[]
Score: {'win': 0, 'loose': 1} 0.0 %


In [13]:
i = 0
historic_pct = []
while True:
    i+=1
    score, data_X, data_y = aprendiendo_jugando(1000, debug=False)
    pct = (score["win"]*100/(score["win"]+score["loose"]))
    historic_pct.append(pct)
    print("Iter: %s - score: %s %s %%" % (i, score, pct))
    
    if len(data_X):
        model = model.partial_fit(data_X, data_y)
    
    if sum(historic_pct[-9:])==900:
        break

Iter: 1 - score: {'win': 563, 'loose': 437} 56.3 %
Iter: 2 - score: {'win': 564, 'loose': 436} 56.4 %
Iter: 3 - score: {'win': 529, 'loose': 471} 52.9 %
Iter: 4 - score: {'win': 541, 'loose': 459} 54.1 %
Iter: 5 - score: {'win': 557, 'loose': 443} 55.7 %
Iter: 6 - score: {'win': 330, 'loose': 670} 33.0 %
Iter: 7 - score: {'win': 324, 'loose': 676} 32.4 %
Iter: 8 - score: {'win': 319, 'loose': 681} 31.9 %
Iter: 9 - score: {'win': 321, 'loose': 679} 32.1 %
Iter: 10 - score: {'win': 324, 'loose': 676} 32.4 %
Iter: 11 - score: {'win': 319, 'loose': 681} 31.9 %
Iter: 12 - score: {'win': 321, 'loose': 679} 32.1 %
Iter: 13 - score: {'win': 336, 'loose': 664} 33.6 %
Iter: 14 - score: {'win': 347, 'loose': 653} 34.7 %
Iter: 15 - score: {'win': 311, 'loose': 689} 31.1 %
Iter: 16 - score: {'win': 324, 'loose': 676} 32.4 %
Iter: 17 - score: {'win': 329, 'loose': 671} 32.9 %
Iter: 18 - score: {'win': 324, 'loose': 676} 32.4 %
Iter: 19 - score: {'win': 337, 'loose': 663} 33.7 %
Iter: 20 - score: {'w

Iter: 158 - score: {'win': 541, 'loose': 459} 54.1 %
Iter: 159 - score: {'win': 547, 'loose': 453} 54.7 %
Iter: 160 - score: {'win': 546, 'loose': 454} 54.6 %
Iter: 161 - score: {'win': 562, 'loose': 438} 56.2 %
Iter: 162 - score: {'win': 562, 'loose': 438} 56.2 %
Iter: 163 - score: {'win': 547, 'loose': 453} 54.7 %
Iter: 164 - score: {'win': 569, 'loose': 431} 56.9 %
Iter: 165 - score: {'win': 570, 'loose': 430} 57.0 %
Iter: 166 - score: {'win': 527, 'loose': 473} 52.7 %
Iter: 167 - score: {'win': 542, 'loose': 458} 54.2 %
Iter: 168 - score: {'win': 548, 'loose': 452} 54.8 %
Iter: 169 - score: {'win': 584, 'loose': 416} 58.4 %
Iter: 170 - score: {'win': 521, 'loose': 479} 52.1 %
Iter: 171 - score: {'win': 545, 'loose': 455} 54.5 %
Iter: 172 - score: {'win': 553, 'loose': 447} 55.3 %
Iter: 173 - score: {'win': 554, 'loose': 446} 55.4 %
Iter: 174 - score: {'win': 534, 'loose': 466} 53.4 %
Iter: 175 - score: {'win': 560, 'loose': 440} 56.0 %
Iter: 176 - score: {'win': 1000, 'loose': 0} 1

In [14]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()

Loading BokehJS ...

In [15]:
x = range(len(historic_pct))
y = historic_pct

p = figure(
    title="Porcetaje de aprendizaje en cada iteración",
    x_axis_label="Iter", y_axis_label="%", width=900)

p.line(x, y, line_width=1)
show(p)

In [16]:
model.predict_proba([paso_lista("piedra")])

array([[0.00860998, 0.00643022, 0.98507972]])